In [9]:
# Library & prepare training, testing data
library(caret)
library(randomForest)
colClasses=c("integer", "factor", "integer", "factor", "integer", "factor", "factor", "factor", "factor", "factor",
             "integer", "integer", "integer", "factor", "factor")

url_train <- "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data_train <- read.table( file=url_train, header=FALSE, colClasses=colClasses, sep=",", strip.white=TRUE )

url_test <- "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
data_test <- read.table( file=url_test, skip=1, header=FALSE, colClasses=colClasses, sep=",", strip.white=TRUE)

#remove trailing dot
data_test[,15] <- factor(sub("\\.", "", data_test[,15]))

# inTrain <- createDataPartition(y=data_train$V15, p=0.7, list=FALSE)
# training <- data_train[inTrain,]
# validation <- data_train[-inTrain,]

In [10]:
# Map levels of training set to test set
common <- intersect(names(data_train), names(data_test)) 
for (p in common) { 
  if (class(data_train[[p]]) == "factor") { 
    levels(data_test[[p]]) <- levels(data_train[[p]]) 
  } 
}

head(data_train,5)
head(data_test,5)



V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,Trinadad&Tobago,<=50K
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,Trinadad&Tobago,<=50K
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,Trinadad&Tobago,>50K
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,Trinadad&Tobago,>50K
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,Trinadad&Tobago,<=50K


In [11]:
# Train model
library(parallel)
library(doParallel)
set.seed(1234)

print(detectCores())
cluster <- makeCluster(detectCores()-1) 
registerDoParallel(cluster)

control <- trainControl(method="cv", number=5, allowParallel = TRUE)
system.time(modFit <- train(V15 ~ ., data=data_train, method="rf", trControl=control))
stopCluster(cluster)
modFit


[1] 4


   user  system elapsed 
 416.11    4.41 4362.81 

Random Forest 

32561 samples
   14 predictor
    2 classes: '<=50K', '>50K' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 26049, 26049, 26049, 26048, 26049 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
    2   0.8096189  0.3092421
   51   0.8571606  0.5873444
  100   0.8553794  0.5836922

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 51.

In [12]:
p <- predict(modFit ,data_test)
confusionMatrix(p, data_test$V15)

Confusion Matrix and Statistics

          Reference
Prediction <=50K  >50K
     <=50K 11733  1635
     >50K    702  2211
                                         
               Accuracy : 0.8565         
                 95% CI : (0.851, 0.8618)
    No Information Rate : 0.7638         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.5658         
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.9435         
            Specificity : 0.5749         
         Pos Pred Value : 0.8777         
         Neg Pred Value : 0.7590         
             Prevalence : 0.7638         
         Detection Rate : 0.7207         
   Detection Prevalence : 0.8211         
      Balanced Accuracy : 0.7592         
                                         
       'Positive' Class : <=50K          
                                         

In [13]:
modFit$finalModel


Call:
 randomForest(x = x, y = y, mtry = param$mtry) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 51

        OOB estimate of  error rate: 14.07%
Confusion matrix:
      <=50K >50K class.error
<=50K 23003 1717  0.06945793
>50K   2865 4976  0.36538707